<a href="https://colab.research.google.com/github/StephenRoddy/training-styletransfer/blob/master/Train_Fast_style_transfer_tensorflow_ml5js.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train and Style Transfer model and run it in ml5.js/tf.js

## 1. Preparing your environment


In [ ]:
# Clone the fast-style-transfer git repo from github: https://github.com/lengstrom/fast-style-transfer.
!git clone https://github.com/lengstrom/fast-style-transfer

Cloning into 'fast-style-transfer'...
remote: Enumerating objects: 235, done.
remote: Total 235 (delta 0), reused 0 (delta 0), pack-reused 235
Receiving objects: 100% (235/235), 11.02 MiB | 44.76 MiB/s, done.
Resolving deltas: 100% (110/110), done.


### Add a style image
1. Go to left sidebar, click on the folder icon to open "Files" panel
2. Create a folder called 'ckpt' inside of 'fast-style-transfer' folder
3. Create another folder called 'images' inside of 'fast-style-transfer' folder
4. Inside of the 'images' folder, create a folder called 'style'
5. Put a style image inside of the 'style' folder

### Install some libraries

In [ ]:
# Install tensorflow
!pip install tensorflow-gpu==2.1.0

     |████████████████████████████████| 421.8MB 28kB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 450kB 55.3MB/s 
     |████████████████████████████████| 3.9MB 47.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=7527536832a7fefe2b57e7e193f6ba5bcb4d5488255dc010ba18ad7dfb8db35b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

In [ ]:
# Install other libraries
!apt install ffmpeg
!pip install moviepy
!pip install scipy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [ ]:
# go to fast-style-transfer 
%cd fast-style-transfer

/content/fast-style-transfer


In [ ]:
%ls

docs.md      examples/  setup.sh*  style.py
evaluate.py  README.md  src/       transform_video.py


# 2. Downloading Datasets

The following step is downloading dataset, it may take 1 hour to finish. Keep this web tab active, and don't close it, wait util the following cell stopped loading.

In [ ]:
!./setup.sh

--2020-10-14 00:01:22--  http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat
Resolving www.vlfeat.org (www.vlfeat.org)... 64.90.48.57
Connecting to www.vlfeat.org (www.vlfeat.org)|64.90.48.57|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat [following]
--2020-10-14 00:01:22--  https://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat
Connecting to www.vlfeat.org (www.vlfeat.org)|64.90.48.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 576042600 (549M)
Saving to: ‘imagenet-vgg-verydeep-19.mat’

imagenet-vgg-veryde 100%[===================>] 549.36M  18.2MB/s    in 31s     

2020-10-14 00:01:54 (17.6 MB/s) - ‘imagenet-vgg-verydeep-19.mat’ saved [576042600/576042600]

--2020-10-14 00:01:54--  http://msvocds.blob.core.windows.net/coco2014/train2014.zip
Resolving msvocds.blob.core.windows.net (msvocds.bl

# 3. Training with style.py
Keep this cell running, keep the tab active and wait. It took me 2 hours.

In [ ]:
# Remember to replace the name after 'images/style/' to your own style image name
# You can change the number after --epoch to change the training time, default to 2, it has to be > 0
!python style.py --checkpoint-dir ckpt --style images/style/pollock.jpg --style-weight 1.5e2 --train-path data/train2014 --vgg-path data/imagenet-vgg-verydeep-19.mat --epochs 2

2020-10-14 00:34:56.765903: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-10-14 00:34:56.766618: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-10-14 00:34:56.766644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Train set has been trimmed slightly..
(1, 500, 750, 3)
2020-10-14 00:35:00.608957: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-14 00

At this point, you will be able to see 4 files in the '/fast-style-transfer/ckpt' folder.

# 4. Converting model to ml5js

In [ ]:
pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 53kB/s 
     |████████████████████████████████| 3.2MB 39.1MB/s 
     |████████████████████████████████| 491kB 59.2MB/s 
ERROR: tensorflow-gpu 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement tensorflow-estimator<2.2.0,>=2.1.0rc0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
%cd ../
!git clone https://github.com/reiinakano/fast-style-transfer-deeplearnjs.git
%cd fast-style-transfer-deeplearnjs

/content
Cloning into 'fast-style-transfer-deeplearnjs'...
remote: Enumerating objects: 1407, done.
remote: Total 1407 (delta 0), reused 0 (delta 0), pack-reused 1407
Receiving objects: 100% (1407/1407), 37.97 MiB | 1.56 MiB/s, done.
Resolving deltas: 100% (135/135), done.
/content/fast-style-transfer-deeplearnjs


In [ ]:
# Change src/ckpts/pollock to src/ckpts/YOUR_OWN_FOLDER
!python scripts/dump_checkpoint_vars.py --output_dir=src/ckpts/pollock --checkpoint_file=../fast-style-transfer/ckpt/fns.ckpt
# Change src/ckpts/pollock to src/ckpts/YOUR_OWN_FOLDER
!python scripts/remove_optimizer_variables.py --output_dir=src/ckpts/pollock

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
!zip -r /content/fast-style-transfer-deeplearnjs/src/ckpts/pollock.zip /content/fast-style-transfer-deeplearnjs/src/ckpts/pollock

  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/ (stored 0%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_47 (stored 0%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_25 (stored 0%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_43 (stored 0%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_13 (stored 0%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable (deflated 7%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_28 (deflated 2%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_40 (stored 0%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_37 (stored 0%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_26 (deflated 4%)
  adding: content/fast-style-transfer-deeplearnjs/src/ckpts/pollock/Variable_23 (deflate

# 5. Run it with ml5js
Now, you should be able to see a 'pollock' .zip file at "fast-style-transfer-deeplearnjs/src/ckpts/pollock.zip". (Sometimes, you need to refresh the Files panel, right click to refresh)

Download the 'pollock' folder and put it into your p5 sketch(https://github.com/yining1023/machine-learning-for-the-web/tree/master/week5-styleTransfer/styleTransfer-ml5/StyleTransfer_Video) under models/.

Change the model path in your p5 sketch: style = ml5.styleTransfer('models/pollock', video, modelLoaded);